In [2]:
import pandas as pd

C:\Users\malco\AppData\Local\Temp\ipykernel_7692\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [31]:
df = pd.read_csv('Yemen.csv', skiprows=[1])
df = df.drop_duplicates()
# Removed unused columns
# df.drop(["market", "latitude", "longitude", "currency", "price", "pricetype", "priceflag", "admin2"], axis = 1, inplace = True) 
df.drop(["market", "latitude", "longitude", "currency", "price", "pricetype", "priceflag", "admin2", "unit"], axis = 1, inplace = True) 



# Rename columns
# df.rename(columns = {"date": "Datetime", "admin1": "AdminStrata","category": "Category", "usdprice" : "Price", "commodity": "Commodity", "unit": "Unit"}, inplace = True)
df.rename(columns = {"date": "Datetime", "admin1": "AdminStrata","category": "Category", "usdprice" : "Price", "commodity": "Commodity"}, inplace = True)

# TODO: consider other categories? 
categories = ['cereals and tubers']
# df = df[df['Category'] != 'non-food']
df = df[df['Category'].isin(categories)]

print(sorted(df["AdminStrata"].unique()))
print(sorted(df["Category"].unique()))
# print(sorted(df["Unit"].unique()))

# Convert date string to datetime object
df["Datetime"] = pd.to_datetime(df["Datetime"])
df.sort_values("Datetime", ascending = True, inplace = True) 
df.head()

['Abyan', 'Aden', 'Al Bayda', "Al Dhale'e", 'Al Hudaydah', 'Al Jawf', 'Al Maharah', 'Al Mahwit', 'Amanat Al Asimah', 'Amran', 'Dhamar', 'Hadramaut', 'Hajjah', 'Ibb', 'Lahj', 'Marib', 'Raymah', "Sa'ada", "Sana'a", 'Shabwah', 'Socotra', 'Taizz']
['cereals and tubers']


,Datetime,AdminStrata,Category,Commodity,Price
0,2009-03-15,Aden,cereals and tubers,Wheat,0.4994
1,2009-03-15,Aden,cereals and tubers,Wheat flour,0.5494
2,2009-03-15,Al Hudaydah,cereals and tubers,Wheat,0.4994
3,2009-03-15,Al Hudaydah,cereals and tubers,Wheat flour,0.5993
4,2009-03-15,Amanat Al Asimah,cereals and tubers,Wheat,0.4495


In [32]:
df = df.groupby(["AdminStrata", "Category", "Commodity"]).apply(lambda x: x.set_index("Datetime").resample("D").mean()).reset_index()
df.reset_index(drop = True, inplace = True)

df = df.set_index(["Datetime", "AdminStrata", "Category", "Commodity"]).unstack(["AdminStrata", "Category", "Commodity"]).sort_index(axis=1).reorder_levels([1,2,3,0], axis=1)
df.sort_index(axis = 1, level = [0, 1, 2], inplace = True)
df.index.freq = "D"
df.head()

AdminStrata              Abyan                                    \
Category    cereals and tubers                                     
Commodity             Potatoes Rice (imported) Wheat Wheat flour   
                         Price           Price Price       Price   
Datetime                                                           
2009-03-15                 NaN             NaN   NaN         NaN   
2009-03-16                 NaN             NaN   NaN         NaN   
2009-03-17                 NaN             NaN   NaN         NaN   
2009-03-18                 NaN             NaN   NaN         NaN   
2009-03-19                 NaN             NaN   NaN         NaN   

AdminStrata               Aden                                      \
Category    cereals and tubers                                       
Commodity             Potatoes Rice (imported)   Wheat Wheat flour   
                         Price           Price   Price       Price   
Datetime                                                             
2009-03-15                 NaN             NaN  0.4994      0.5494   
2009-03-16                 NaN             NaN     NaN         NaN   
2009-03-17                 NaN             NaN     NaN         NaN   
2009-03-18                 NaN             NaN     NaN         NaN   
2009-03-19                 NaN             NaN     NaN         NaN   

AdminStrata           Al Bayda                  ...            Shabwah  \
Category    cereals and tubers                  ... cereals and tubers   
Commodity             Potatoes Rice (imported)  ...              Wheat   
                         Price           Price  ...              Price   
Datetime                                        ...                      
2009-03-15                 NaN             NaN  ...                NaN   
2009-03-16                 NaN             NaN  ...                NaN   
2009-03-17                 NaN             NaN  ...                NaN   
2009-03-18                 NaN             NaN  ...                NaN   
2009-03-19                 NaN             NaN  ...                NaN   

AdminStrata                        Socotra                          \
Category                cereals and tubers                           
Commodity   Wheat flour           Potatoes Rice (imported)   Wheat   
                  Price              Price           Price   Price   
Datetime                                                             
2009-03-15          NaN                NaN             NaN  0.4994   
2009-03-16          NaN                NaN             NaN     NaN   
2009-03-17          NaN                NaN             NaN     NaN   
2009-03-18          NaN                NaN             NaN     NaN   
2009-03-19          NaN                NaN             NaN     NaN   

AdminStrata                          Taizz                                    
Category                cereals and tubers                                    
Commodity   Wheat flour           Potatoes Rice (imported) Wheat Wheat flour  
                  Price              Price           Price Price       Price  
Datetime                                                                      
2009-03-15       0.5993                NaN             NaN   NaN      0.5494  
2009-03-16          NaN                NaN             NaN   NaN         NaN  
2009-03-17          NaN                NaN             NaN   NaN         NaN  
2009-03-18          NaN                NaN             NaN   NaN         NaN  
2009-03-19          NaN                NaN             NaN   NaN         NaN  

[5 rows x 98 columns]

In [ ]:
print(df[df['Category'] == 'cereals and tubers'].shape[0])
print(df[df['Category'] == 'meat, fish and eggs'].shape[0])
print(df[df['Category'] == 'pulses and nuts'].shape[0])
print(df[df['Category'] == 'vegetables and fruits'].shape[0])

In [ ]:
print(sorted(df['Commodity'].unique()))
for commodity in sorted(df['Commodity'].unique()):
    print(f"{commodity} - {df[df['Commodity'] == commodity].shape[0]}")

In [36]:
# Interpolate data to get daily frequency

start_index = pd.to_datetime('2009-01-01')
end_index = pd.to_datetime('2024-12-31')

def get_start_end(x): 
    # TODO split indexing part to own function
    global start_index
    global end_index
    first_val_index = x.apply(lambda col: col.first_valid_index()).max()
    last_val_index = x.last_valid_index()

    if first_val_index > start_index:
        start_index = first_val_index
    if last_val_index < end_index:
        end_index = last_val_index
    # ========================================
        

   
    return x

def clip(x):
    global start_index
    global end_index
    return x.loc[start_index:end_index]

# select commodities where data is available for all adminstrata
def limit_commodity(x):
    global commodity_intersection
    # print(x.columns.get_level_values("Commodity"))
    # print(x.columns.get_level_values("Commodity").isin(set.intersection(*map(set, commodity_intersection))))
    return x.iloc[:, x.columns.get_level_values("Commodity").isin(set.intersection(*map(set, commodity_intersection)))]

df_interpolate = df.groupby(axis = 1, level = "AdminStrata", group_keys = False).apply(lambda x: x.interpolate(method="linear", limit=90))
df_interpolate = df_interpolate.groupby(axis = 1, level = "AdminStrata", group_keys = False).apply(get_start_end).apply(clip)


commodity_intersection = list(df_interpolate.groupby(axis = 1, level = "AdminStrata").apply(lambda x: x.columns.get_level_values("Commodity")))
print(set.intersection(*map(set, commodity_intersection)))

# df_interpolate = df_interpolate.groupby(axis = 1, level = "AdminStrata").apply(limit_commodity)

# print(df.columns.get_level_values("Commodity"))
# print(df.columns.get_level_values("Commodity").isin(commodity_intersection))
# print(df.columns)
# print(f"start: {start_index}")
# print(f"end: {end_index}")
df_interpolate.head()

C:\Users\malco\AppData\Local\Temp\ipykernel_7692\1167447422.py:39: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_interpolate = df.groupby(axis = 1, level = "AdminStrata", group_keys = False).apply(lambda x: x.interpolate(method="linear", limit=90))
C:\Users\malco\AppData\Local\Temp\ipykernel_7692\1167447422.py:41: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_interpolate = df_interpolate.groupby(axis = 1, level = "AdminStrata", group_keys = False).apply(get_start_end).apply(clip)


{'Wheat flour', 'Rice (imported)', 'Wheat', 'Potatoes'}


C:\Users\malco\AppData\Local\Temp\ipykernel_7692\1167447422.py:44: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  commodity_intersection = list(df_interpolate.groupby(axis = 1, level = "AdminStrata").apply(lambda x: x.columns.get_level_values("Commodity")))


AdminStrata              Abyan                                        \
Category    cereals and tubers                                         
Commodity             Potatoes Rice (imported)     Wheat Wheat flour   
                         Price           Price     Price       Price   
Datetime                                                               
2016-10-15            1.080600        1.516765  0.517800    0.528200   
2016-10-16            1.084490        1.515370  0.520729    0.532345   
2016-10-17            1.088381        1.513974  0.523658    0.536490   
2016-10-18            1.092271        1.512578  0.526587    0.540635   
2016-10-19            1.096161        1.511183  0.529516    0.544781   

AdminStrata               Aden                                        \
Category    cereals and tubers                                         
Commodity             Potatoes Rice (imported)     Wheat Wheat flour   
                         Price           Price     Price       Price   
Datetime                                                               
2016-10-15            1.196530        1.501800  0.528200    0.607600   
2016-10-16            1.197970        1.511606  0.531439    0.610861   
2016-10-17            1.199411        1.521413  0.534677    0.614123   
2016-10-18            1.200852        1.531219  0.537916    0.617384   
2016-10-19            1.202293        1.541026  0.541155    0.620645   

AdminStrata           Al Bayda                  ...            Shabwah  \
Category    cereals and tubers                  ... cereals and tubers   
Commodity             Potatoes Rice (imported)  ...              Wheat   
                         Price           Price  ...              Price   
Datetime                                        ...                      
2016-10-15            1.275316        1.676554  ...           0.414300   
2016-10-16            1.279384        1.679046  ...           0.416694   
2016-10-17            1.283451        1.681538  ...           0.419087   
2016-10-18            1.287518        1.684030  ...           0.421481   
2016-10-19            1.291585        1.686521  ...           0.423874   

AdminStrata                        Socotra                            \
Category                cereals and tubers                             
Commodity   Wheat flour           Potatoes Rice (imported)     Wheat   
                  Price              Price           Price     Price   
Datetime                                                               
2016-10-15     0.856200           2.761800        1.280800  0.690500   
2016-10-16     0.861774           2.776039        1.282782  0.688372   
2016-10-17     0.867348           2.790277        1.284764  0.686244   
2016-10-18     0.872923           2.804516        1.286746  0.684116   
2016-10-19     0.878497           2.818755        1.288728  0.681989   

AdminStrata                          Taizz                            \
Category                cereals and tubers                             
Commodity   Wheat flour           Potatoes Rice (imported)     Wheat   
                  Price              Price           Price     Price   
Datetime                                                               
2016-10-15     0.725000           1.320533        1.817848  0.707700   
2016-10-16     0.726671           1.319434        1.818104  0.709413   
2016-10-17     0.728342           1.318335        1.818361  0.711126   
2016-10-18     0.730013           1.317236        1.818617  0.712839   
2016-10-19     0.731684           1.316137        1.818874  0.714552   

AdminStrata              
Category                 
Commodity   Wheat flour  
                  Price  
Datetime                 
2016-10-15     0.880300  
2016-10-16     0.883935  
2016-10-17     0.887571  
2016-10-18     0.891206  
2016-10-19     0.894842  

[5 rows x 98 columns]